In [2]:
import h5py

In [3]:
import numpy as np

In [48]:
import pandas as pd
import anndata as ad
import scanpy as sc
from itertools import product

In [5]:
import scipy

In [6]:
data_path = '/data_volume/memento/'

In [7]:
adata = sc.read('Data/fullHBEC.h5ad')

In [8]:
adata.X

<69958x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 216759439 stored elements in Compressed Sparse Row format>

In [9]:
type(adata)

anndata._core.anndata.AnnData

In [10]:
index=adata.var_names

In [11]:
index_df=pd.DataFrame(index)

In [92]:
adata.var

,gene_ids,feature_types,genome,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts
HES4,ENSG00000188290,Gene Expression,GRCh38,False,67040,5.536824,16.879510,446567.0
ISG15,ENSG00000187608,Gene Expression,GRCh38,False,78984,52.471233,2.070573,4232015.0
TTLL10,ENSG00000162571,Gene Expression,GRCh38,False,20165,0.626987,74.998140,50569.0
SDF4,ENSG00000078808,Gene Expression,GRCh38,False,28066,0.463511,65.201974,37384.0
AURKAIP1,ENSG00000175756,Gene Expression,GRCh38,False,61429,2.226883,23.836388,179607.0
...,...,...,...,...,...,...,...,...
RPL10,ENSG00000147403,Gene Expression,GRCh38,False,77640,43.090015,3.736950,3475382.0
AC244090.1,ENSG00000197180,Gene Expression,GRCh38,False,19842,0.621866,75.398616,50156.0
ATP6AP1,ENSG00000071553,Gene Expression,GRCh38,False,32831,0.629752,59.294021,50792.0
FAM50A,ENSG00000071859,Gene Expression,GRCh38,False,30762,0.547648,61.859300,44170.0


In [12]:
index_df.to_csv('all_variables.csv')

In [13]:
index2=adata.obs_names
index_df2=pd.DataFrame(index2)
index_df2.to_csv('all_index.csv')

In [14]:
print(adata.var.index.str.contains('MT-'))

[False False False ... False False False]


In [15]:
adata.obs['time'].unique

<bound method Series.unique of AAACCCAAGGAAGTAG-1     9
AAACCCAAGGGACTGT-1    24
AAACCCACAATCTGCA-1     3
AAACCCACACGTTCGG-1     3
AAACCCACAGAACTAA-1    48
                      ..
TTTGTTGGTATTGACC-4    24
TTTGTTGGTCGACTTA-4     3
TTTGTTGGTCGCTGCA-4     0
TTTGTTGTCCGGTAAT-4     9
TTTGTTGTCGGACCAC-4    48
Name: time, Length: 69958, dtype: category
Categories (6, object): ['0', '3', '6', '9', '24', '48']>

In [16]:
mean_filter = adata.X.mean(axis=0).A1>0.5
mt_filter  =~adata.var.index.str.contains('MT-')
gene_filter = mean_filter & mt_filter

In [94]:
adata

View of AnnData object with n_obs × n_vars = 3126 × 2991
    obs: 'NUM.SNPS', 'BEST.GUESS', 'DROPLET.TYPE', 'batch', 'HTO_classification', 'condition', 'donor', 'stim', 'time', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [93]:
adata = adata[(adata.obs['stim']=='lambda') &  adata.obs['time'].isin(['0','3','6','9','24']) & adata.obs['cell_type'].isin(['ciliated']), gene_filter]

IndexError: Boolean index does not match AnnData’s shape along this dimension. Boolean index has shape (36601,) while AnnData index has shape (2991,).

In [19]:
df_cond=adata.obs[[ 'cell_type', 'time']]
df_gene= pd.DataFrame(data=adata.X.toarray(), index=adata.obs.index, columns=adata.var.index)

In [46]:
df_718 = pd.concat([df_cond, df_gene], axis=1)
df_718.to_csv('718table.csv')

In [90]:
#different tables with times 
all_cell_types = ['ciliated']
all_time=['3','6','9','24'] #0 is empty and has Nan
combo=list(product(all_cell_types,all_time))

list_mean_cxlxt=[]
for i,j in combo:
    df_cxlxt=df_718.loc[(df_718.cell_type==i)& (df_718.time==j),:]
    cilxlambda=df_cxlxt.drop(['time','cell_type'], axis=1)
    norm_cxl=cilxlambda.div(cilxlambda.sum(axis=1), axis=0) #normalize
    norm_cxl=np.array(norm_cxl)
    norm_cxl=np.log1p(norm_cxl)  #log1p
    norm_cxl=pd.DataFrame(norm_cxl,index=cilxlambda.index, columns= cilxlambda.columns) #add index and col back
    mean_cxl=norm_cxl.mean() #mean
    list_mean_cxlxt.append(mean_cxl)
    
print(list_mean_cxlxt)


[HES4          0.000304
ISG15         0.002640
TTLL10        0.000428
SDF4          0.000065
AURKAIP1      0.000422
                ...   
RPL10         0.002215
AC244090.1    0.000349
ATP6AP1       0.000177
FAM50A        0.000114
RPS4Y1        0.000282
Length: 2991, dtype: float32, HES4          0.000314
ISG15         0.003063
TTLL10        0.000415
SDF4          0.000068
AURKAIP1      0.000409
                ...   
RPL10         0.002587
AC244090.1    0.000295
ATP6AP1       0.000161
FAM50A        0.000115
RPS4Y1        0.000162
Length: 2991, dtype: float32, HES4          0.000331
ISG15         0.002902
TTLL10        0.000387
SDF4          0.000061
AURKAIP1      0.000423
                ...   
RPL10         0.002760
AC244090.1    0.000334
ATP6AP1       0.000160
FAM50A        0.000114
RPS4Y1        0.000332
Length: 2991, dtype: float32, HES4          0.000339
ISG15         0.002922
TTLL10        0.000342
SDF4          0.000061
AURKAIP1      0.000402
                ...   
RPL10       

In [76]:
#final mean table
names = [f'{i}-{j}' for i,j in combo]
df_mean_cxlxt=pd.DataFrame(list_mean_cxlxt, index=names)
df_mean_cxlxt.

,HES4,ISG15,TTLL10,SDF4,AURKAIP1,MRPL20,SSU72,GNB1,FAAP20,RER1,...,SLC6A8,BCAP31,IDH3G,SSR4,NAA10,RPL10,AC244090.1,ATP6AP1,FAM50A,RPS4Y1
ciliated-3,0.000304,0.002640,0.000428,0.000065,0.000422,0.000353,0.000199,0.000144,0.000134,0.000228,...,0.000046,0.000207,0.000103,0.000535,0.000111,0.002215,0.000349,0.000177,0.000114,0.000282
ciliated-6,0.000314,0.003063,0.000415,0.000068,0.000409,0.000344,0.000200,0.000140,0.000136,0.000210,...,0.000045,0.000206,0.000108,0.000554,0.000124,0.002587,0.000295,0.000161,0.000115,0.000162
ciliated-9,0.000331,0.002902,0.000387,0.000061,0.000423,0.000359,0.000199,0.000136,0.000131,0.000241,...,0.000054,0.000210,0.000110,0.000589,0.000117,0.002760,0.000334,0.000160,0.000114,0.000332
ciliated-24,0.000339,0.002922,0.000342,0.000061,0.000402,0.000357,0.000203,0.000121,0.000128,0.000225,...,0.000035,0.000206,0.000109,0.000593,0.000120,0.002940,0.000334,0.000152,0.000111,0.000226


In [95]:
#difference in means
names=[f'{i}-{j}' for i, j in combo]
names1=list(names)
for x,y in names1:
    df_meandiff_cxlxt=df_mean_cxlxt.loc[x,:]-df_mean_cxlxt.loc[y,:]

ValueError: too many values to unpack (expected 2)

HES4          0.0
ISG15         0.0
TTLL10        0.0
SDF4          0.0
AURKAIP1      0.0
             ... 
RPL10         0.0
AC244090.1    0.0
ATP6AP1       0.0
FAM50A        0.0
RPS4Y1        0.0
Name: ciliated-24, Length: 2991, dtype: float32